In [1]:
# Importação das bibliotecas básicas
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

In [2]:
# Importando as funções do arquivo "crypto_strategy.py"
# Certifique-se de que o arquivo "crypto_strategy.py" está no mesmo diretório do notebook
from cryptostrategy import coletar_dados,calcular_indicadores,calcular_atr,calcular_todos_indicadores,gerar_sinais,calcular_retorno_total,otimizar_janela,determinar_tendencia_e_stoploss


In [3]:
# Lista de tickers para criptomoedas (exemplo)
tickers_crypto = ['ONE-USD', 'ALICE-USD', 'ADA-USD','FLOW-USD','GALA-USD','RVN-USD','FIL-USD','LUNA1-USD']

In [ ]:



# Dicionário para armazenar os resultados
resultados_tendencia = {}
tickers_crypto = sorted(tickers_crypto)
for ticker in tickers_crypto:
    print(f"\nProcessando {ticker}...")
    # Coleta os dados históricos para o ticker (passando como lista: [ticker])
    dados_crypto = coletar_dados([ticker], start_date='2010-01-01', end_date=datetime.date.today())
    df = dados_crypto[ticker].droplevel('Ticker',axis=1)

    if df.empty:
        print(f"Aviso: Nenhum dado encontrado para {ticker}.")
        continue

    ultimo_preco = df.iloc[-1]['Close']

    
    # Otimização de janela (por exemplo, janelas de 10 a 30 dias)
    #print(f"\nOtimização da janela para {ticker}...")
    resultados_janela = otimizar_janela(df, janela_min=1, janela_max=99)
    
    # Identifica a melhor janela (a primeira linha do DataFrame ordenado)
    melhor_janela = int(resultados_janela.iloc[0]['Janela'])
    melhor_retorno = resultados_janela.iloc[0]['Retorno_Total']
    #print(f"Melhor janela para {ticker}: {melhor_janela} dias (Retorno Total: {melhor_retorno:.2f})")
    
    # Recalcula os indicadores e sinais com a melhor janela
    _, df_sinais = calcular_retorno_total(df, melhor_janela)
    
    # Determina a tendência atual e, se houver reversão, o stop loss
    tendencia, posicao_revertida, stop_loss = determinar_tendencia_e_stoploss(df_sinais)
    
    # Armazena o resultado no dicionário
#    resultados_tendencia[ticker] = (tendencia, posicao_revertida, stop_loss, melhor_janela, melhor_retorno)
    resultados_tendencia[ticker] = (tendencia, posicao_revertida, stop_loss, melhor_janela, melhor_retorno, ultimo_preco)
    
    if posicao_revertida is None:
        print(f"{ticker}: Tendência atual: {tendencia}. Último preço: {ultimo_preco:.4f}")
    else:
        if posicao_revertida == "Long":
            print(f"{ticker}: Reversão de posição Long detectada. Tendência atual: {tendencia}. Stop Loss sugerido: {stop_loss:.4f}. Último preço: {ultimo_preco:.4f}")
        elif posicao_revertida == "Short":
            print(f"{ticker}: Reversão de posição Short detectada. Tendência atual: {tendencia}. Stop Loss sugerido: {stop_loss:.4f}. Último preço: {ultimo_preco:.4f}")

# Ao final, itere pelo dicionário 'resultados_tendencia' para consolidar os avisos
print("\nResumo das Tendências e Otimizações:")
for ticker, (tendencia, posicao, sl, janela, ret, ultimo_preco) in resultados_tendencia.items():
    if posicao is None:
        print(f"{ticker}: Tendência: {tendencia}. Janela Ótima: {janela} dias (Retorno: {ret:.4f}). Último preço: {ultimo_preco:.4f}")
    else:
        if posicao == "Long":
            print(f"{ticker}: Reversão de Long. Tendência: {tendencia}. Stop Loss: {sl:.4f}. Janela Ótima: {janela} dias (Retorno: {ret:.4f}). Último preço: {ultimo_preco:.4f}")
        else:
            print(f"{ticker}: Reversão de Short. Tendência: {tendencia}. Stop Loss: {sl:.4f}. Janela Ótima: {janela} dias (Retorno: {ret:.4f}). Último preço: {ultimo_preco:.4f}")



Processando ADA-USD...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
